In [39]:
import cv2
import numpy as np
import json
import os
from numba import jit
import struct

In [40]:
# 生成出D_best的所有仿射變換矩陣
@jit
def affine_transformation(D_block,method):
    match method:
        case 0:
            # 無轉換
            return D_block
        case 1:
            # 左右翻轉
            return np.fliplr(D_block)
        case 2:
            # 上下翻轉
            return np.flipud(D_block)
        case 3:
            # 主對角對稱
            return D_block.T
        case 4:
            # 次對角對稱
            return np.flip(D_block.T)
        case 5:
            # 旋轉90度
            return np.rot90(D_block,1)
        case 6:
            # 旋轉180度
            return np.rot90(D_block,2)
        case 7:
            # 旋轉270度
            return np.rot90(D_block,3)
        case _:
            return D_block
        
# 做線性回歸，找到一組s跟o，讓損失函數越小越好 [R-(sD+o)]
@jit
def compute_SO(rangeSize,R_block,D_block):
    meanR=np.sum(R_block)/(rangeSize**2)
    meanD=np.sum(D_block)/(rangeSize**2)

    s=np.sum((R_block-meanR)*(D_block-meanD))/np.sum((D_block-meanD)**2)
    o=int(meanR-s*meanD)
    # 至小數點後 2 位
    s=round(s,2)

    # if s>1:
    #     s=0
    # elif s<0.25:
    #     s=0.25
    # elif s<0.5:
    #     s=0.5
    # elif s<0.75:
    #     s=0.75
    # else:
    #     s=1

    # if o>128:
    #     o=128

    return s,o

# 計算與Ri差異
@jit
def compute_loss(R,D,s,o):
    loss = R-(D*s+o)
    sum = 0
    for row in range(loss.shape[0]):
        for col in range(loss.shape[1]):
            sum+=abs(loss[row][col])
    return sum

# 找到與Ri最相似的D塊
@jit
def find_suitable_block(rangeSize,R_block,Domain_Blocks):
    # 依序算出每個D塊與Ri的差異，並找到差異最小的D塊D_best
    min_loss= float(1e9)
    for row in range(Domain_Blocks.shape[0]):
        for col in range(Domain_Blocks.shape[1]):
            # D_best全部的8種仿射變換矩陣中挑選出與Ri最相似的轉換
            for i in range(8):
                D=affine_transformation(Domain_Blocks[row][col],i)
                s,o=compute_SO(rangeSize,R_block,D)
                loss=compute_loss(R_block,D,s,o)
                if  loss < min_loss:
                    min_loss = loss
                    param=[row,col,i,s,o]
    
    return param
    

C:\Users\jush\AppData\Local\Temp\ipykernel_18956\2173216202.py:3: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def affine_transformation(D_block,method):
C:\Users\jush\AppData\Local\Temp\ipykernel_18956\2173216202.py:34: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def compute_SO(rangeSize,R_block,D_block):
C:\Users\jush\AppData\Local\Temp\ipykernel_18956\2173

In [41]:
class Compressor():
    def __init__(self,file,rangeSize,domainSize,stepSize):
        self.file = file
        self.rangeSize = rangeSize
        self.domainSize = domainSize
        self.stepSize = stepSize
        self.output=[]
    
        # 壓縮 輸出格式 = D索引(2維) , 轉換方法(0~7)  , s(0,0.25,0.5,0.75,1) , o(0~128)
        self.root , extension = os.path.splitext(self.file)
        if extension == '.gif':
            cap = cv2.VideoCapture(self.file)
            _, img = cap.read()
            cap.release()
        else:
            # 使用cv2.imread讀取是BGR格式
            img=cv2.imread(self.file)
        # 轉成灰階圖
        self.input=cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)

        # 紀錄圖片大小以及壓縮資訊
        # min_block[0] = 高 , 寬 , rangeSize , domainSize , stepSize
        self.output.append([self.input.shape[0],self.input.shape[1],self.rangeSize,self.domainSize,self.stepSize])
        
    # 將圖片分成R塊(destination blocks)跟D塊(source blocks)
    # R是不可重疊的區塊
    # D是可重疊區塊。每隔步長S取一個D塊。假設圖像大小L*L，D大小爲B*B，則D會有((L-B)/S+1)*((L-B)/S+1)塊。
    def generate_RD(self):
        # 將圖片拆成4x4的Range Block
        range_Blocks=np.empty((self.input.shape[0]//self.rangeSize,self.input.shape[1]//self.rangeSize,self.rangeSize,self.rangeSize),np.uint8)
        for row in range(self.input.shape[0]//self.rangeSize):
            for col in range(self.input.shape[1]//self.rangeSize):
                range_Blocks[row,col]=self.input[row*self.rangeSize:(row+1)*self.rangeSize,col*self.rangeSize:(col+1)*self.rangeSize].copy()

        # 將圖片拆成8x8的Domain Block
        domain_Blocks=np.empty(((self.input.shape[0]-self.domainSize)//self.stepSize+1,(self.input.shape[1]-self.domainSize)//self.stepSize+1,self.domainSize,self.domainSize),np.uint8)
        for row in range(domain_Blocks.shape[0]):
            for col in range(domain_Blocks.shape[1]):
                domain_Blocks[row,col]=self.input[row*self.stepSize:row*self.stepSize+self.domainSize,col*self.stepSize:col*self.stepSize+self.domainSize].copy()

        # Domain Block均值收縮成Range Block大小
        domainContraction_Blocks=np.empty((domain_Blocks.shape[0],domain_Blocks.shape[1],self.rangeSize,self.rangeSize),np.uint8)
        for row in range(domain_Blocks.shape[0]):
            for col in range(domain_Blocks.shape[1]):
                domainContraction_Blocks[row,col]=cv2.resize(domain_Blocks[row,col],(self.rangeSize,self.rangeSize))

        return range_Blocks,domainContraction_Blocks
    
    def compress(self):
        # 定義Range_Blocks跟Domain_Blocks
        Range_Blocks,Domain_Blocks=self.generate_RD()
        
        # 依序處理R塊
        for row in range(Range_Blocks.shape[0]):
            self.output.append([])
            for col in range(Range_Blocks.shape[1]):
                print("R:",row,col)

                # 找到與Ri最相似的D塊
                [r,c,method,s,o]=find_suitable_block(self.rangeSize,Range_Blocks[row][col],Domain_Blocks)
                
                # 紀錄壓縮後的資料 (D塊編號,仿射變換方法,contrast跟brightness)
                self.output[-1].append([int(r),int(c),int(method),s,int(o)])


        # 將壓縮結果輸出成json檔
        with open("output/"+self.root+".json", 'w') as file:
            json.dump(self.output, file)


In [42]:
compressor=Compressor("house.jpg",4,8,4)
compressor.compress()

R: 0 0
R: 0 1
R: 0 2
R: 0 3
R: 0 4
R: 0 5
R: 0 6
R: 0 7
R: 0 8
R: 0 9
R: 0 10
R: 0 11
R: 0 12
R: 0 13
R: 0 14
R: 0 15
R: 0 16
R: 0 17
R: 0 18
R: 0 19
R: 0 20
R: 0 21
R: 0 22
R: 0 23
R: 0 24
R: 0 25
R: 0 26
R: 0 27
R: 0 28
R: 0 29
R: 0 30
R: 0 31
R: 1 0
R: 1 1
R: 1 2
R: 1 3
R: 1 4
R: 1 5
R: 1 6
R: 1 7
R: 1 8
R: 1 9
R: 1 10
R: 1 11
R: 1 12
R: 1 13
R: 1 14
R: 1 15
R: 1 16
R: 1 17
R: 1 18
R: 1 19
R: 1 20
R: 1 21
R: 1 22
R: 1 23
R: 1 24
R: 1 25
R: 1 26
R: 1 27
R: 1 28
R: 1 29
R: 1 30
R: 1 31
R: 2 0
R: 2 1
R: 2 2
R: 2 3
R: 2 4
R: 2 5
R: 2 6
R: 2 7
R: 2 8
R: 2 9
R: 2 10
R: 2 11
R: 2 12
R: 2 13
R: 2 14
R: 2 15
R: 2 16
R: 2 17
R: 2 18
R: 2 19
R: 2 20
R: 2 21
R: 2 22
R: 2 23
R: 2 24
R: 2 25
R: 2 26
R: 2 27
R: 2 28
R: 2 29
R: 2 30
R: 2 31
R: 3 0
R: 3 1
R: 3 2
R: 3 3
R: 3 4
R: 3 5
R: 3 6
R: 3 7
R: 3 8
R: 3 9
R: 3 10
R: 3 11
R: 3 12
R: 3 13
R: 3 14
R: 3 15
R: 3 16
R: 3 17
R: 3 18
R: 3 19
R: 3 20
R: 3 21
R: 3 22
R: 3 23
R: 3 24
R: 3 25
R: 3 26
R: 3 27
R: 3 28
R: 3 29
R: 3 30
R: 3 31
R: 4 0
R: 4 1
R:

In [11]:
cv2.imwrite("house.jpg",cv2.resize(cv2.imread("pexels-alex-brandes-1196758.jpg"), (128, 128)))

True

In [13]:
def pack_bits(s, o, a):
    return (a << 12) | (o << 5) | s

def unpack_bits(merged_data):
    s = merged_data & 0b11111
    o = (merged_data >> 5) & 0b1111111
    a = (merged_data >> 12) & 0b11
    return s, o, a

s = 0b10101  # 5 bits
o = 0b1101011  # 7 bits
a = 0b10  # 2 bits

# 合併s、o和a為位元串
merged_data = pack_bits(s, o, a)

# 儲存成二進位檔案
with open('data.bin', 'wb') as file:
    # 將位元串轉換為位元組串並寫入檔案
    packed_data = merged_data.to_bytes((merged_data.bit_length() + 7) // 8, byteorder='big')
    file.write(packed_data)

# 從檔案讀取並解包位元串
with open('data.bin', 'rb') as file:
    packed_data = file.read()

# 將位元組串轉換為位元串
merged_data_read = int.from_bytes(packed_data, byteorder='big')

# 解包位元串
s_read, o_read, a_read = unpack_bits(merged_data_read)

print(f"s: {s_read}, o: {o_read}, a: {a_read}")


s: 21, o: 107, a: 2
